# Deep Learning / Linear Algebra / Numpy / PyTorch


## L2 (Euclidean) Distance / Broadcast / Dot Product
$$
(a-b)^2 = a^2-2ab+b^2
$$ 
(Matrix Version)

In [17]:
import numpy as np

X_test = np.array([[1, 2], [3, 4]])
X_train = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
# reshape to broadcast each test example for all training examples
sqr_1 = np.reshape(np.sum(X_test ** 2, axis=1), (X_test.shape[0], 1))
sqr_2 = np.sum(X_train ** 2, axis=1)
print('Square 1: \n', sqr_1)
print('Square 2: \n', sqr_2)
print('Sum of squares: \n', sqr_1 + sqr_2)

# dot product of the test set and the training set's transposed matrix
multi = np.matmul(X_test, X_train.T)
_multi = np.dot(X_test, X_train.T)
l2_dist = np.sqrt(sqr_1 - 2 * multi + sqr_2)
print('Multipliction: \n', multi)
print('Dot product: \n', _multi)
print('L2 distance: \n', l2_dist)

Square 1: 
 [[ 5]
 [25]]
Square 2: 
 [  5  25  61 113]
Sum of squares: 
 [[ 10  30  66 118]
 [ 30  50  86 138]]
Multipliction: 
 [[ 5 11 17 23]
 [11 25 39 53]]
Dot product: 
 [[ 5 11 17 23]
 [11 25 39 53]]
L2 distance: 
 [[0.         2.82842712 5.65685425 8.48528137]
 [2.82842712 0.         2.82842712 5.65685425]]


## SVM Classifier / Hinge Loss
The example of the SVM loss function for a single datapoint:
$$
L_{i}=\sum_{j \neq y_{i}}\left[\max \left(0, w_{j}^{T} x_{i}-w_{y_{i}}^{T} x_{i}+\Delta\right)\right]
$$
Taking the gradient with respect to $w_{y_i}$ we obtain:
$$
\nabla_{w_{y_{i}}} L_{i}=-\left(\sum_{j \neq y_{i}} \mathbb{1}\left(w_{j}^{T} x_{i}-w_{y_{i}}^{T} x_{i}+\Delta>0\right)\right) x_{i}
$$
(when you’re implementing this in code you’d simply count the number of classes that didn’t meet the desired margin (and hence contributed to the loss function) and then the data vector $x_i$ scaled by this number is the gradient.)

For the other rows where $j\neq y_i$ the gradient is:
$$
\nabla_{w_{j}} L_{i}=\mathbb{1}\left(w_{j}^{T} x_{i}-w_{y_{i}}^{T} x_{i}+\Delta>0\right) x_{i}
$$

In [7]:
import numpy as np
from numpy import ndarray

regularization = 5e-7

X: ndarray = np.array([[1,2], [3, 4], [5, 6], [7, 8]])
y: ndarray = np.array([1, 0, 2, 1])

num_train, dim = X.shape
num_classes = np.max(y) + 1

W: ndarray = 0.001 * np.random.randn(dim, num_classes)
dW: ndarray = np.zeros(W.shape)

## Hinge Loss
# linear classifier
scores: ndarray = X.dot(W)
# scores of correct class
correct_class_score: ndarray = scores[np.arange(num_train), y]
# reshape the correct_class_score in order to broadcast to the shape of scores
reshaped_correct_class_score: ndarray = correct_class_score.reshape((num_train, 1))
# margin
margin: ndarray = np.maximum(0, scores - reshaped_correct_class_score + 1)
margin[np.arange(num_train), y] = 0     # correct class margin
# data loss and regularization loss
loss = np.sum(margin) / num_train + regularization * np.sum(W * W)
# print('Scores: \n', scores)
# print('Correct_class_scores: \n', correct_class_score)
# print('Reshaped_correct_class_scores: \n', reshaped_correct_class_score)
# print('Margin: \n', margin)
print('Loss: \n', loss)
print()

## Gradient
# a mask to add certain X_i to certain dW[:, j]
mask: ndarray = np.zeros(margin.shape)
# gradient of incorrect classes: X_i when margin > 0 
mask[margin > 0] = 1
# gradient of correct classes: -k * X_i where k is the number of classes with margin > 0
mask[np.arange(num_train), y] -= np.sum(mask, axis=1)
# add k * X_i to dW[:, j] where k = mask_ij
data_loss_gradient: ndarray = X.T.dot(mask) / num_train
# data loss gradient and regularization loss gradient
dW = data_loss_gradient + 2 * regularization * W
# print('Mask: \n', mask)
print('Gradient: \n', dW)

Loss: 
 2.0069131264939335

Gradient: 
 [[ 1.75 -2.    0.25]
 [ 2.   -2.5   0.5 ]]


## Softmax Classifier / Cross-entropy Loss
Scores: 
$$
f = WX
$$

Softmax Function:
$$
p_i = \frac{e^{f_i}}{\sum_{j} e^{f_{j}}}
$$

Loss Function:
$$
L_i = -\log p_{y_i}
$$

Gradient Function:
$$
\nabla_{w_j}L_i = \sum_i (p_i-\mathbb{1}(i = y_i))

In [3]:
import numpy as np
from numpy import ndarray

regularization = 5e-7

X: ndarray = np.array([[1,2], [3, 4], [5, 6], [7, 8]])
y: ndarray = np.array([1, 0, 2, 1])

num_train, dim = X.shape
num_classes = np.max(y) + 1

W: ndarray = 0.001 * np.random.randn(dim, num_classes)
dW: ndarray = np.zeros(W.shape)

scores = X.dot(W)
# memo: numeric stability to avoid dividing large numbers due to the exponential
scores -= np.max(scores, axis=1, keepdims=True)

# memo: softmax
probs = np.exp(scores)
sum_probs = np.sum(probs, axis=1, keepdims=True)
softmax = np.divide(probs, sum_probs)

# memo: data loss and regularization loss
loss = -np.sum(np.log(softmax[np.arange(num_train), y])) / num_train \
       + regularization * np.sum(W * W)

# memo: mask for gradient calculation
mask = np.copy(softmax)
mask[np.arange(num_train), y] -= 1

# memo: data gradient and regularization gradient
dW = X.T.dot(mask) / num_train + 2 * regularization * W

print(loss, '\n', dW)

1.0971536661606218 
 [[ 0.57449087 -0.65888945  0.08439858]
 [ 0.65602617 -0.82397117  0.16794501]]


## **Linear Algebra**


### Gradients for vectorized operations
##### Matrix-Matrix multiply gradient
Tip: use dimension analysis! Note that you do not need to remember the expressions for `dX` and `dW` because they are easy to re-derive based on dimensions. For instance, we know that the gradient on the weights `dX` must be of the same size as `X` after it is computed, and that it must depend on matrix multiplication of `W` and `dS` (as is the case when both `W,X` are single numbers and not matrices). There is always exactly one way of achieving this so that the dimensions work out. For example, `W` is of size [10 x 3] and `dS` of size [5 x 3], so if we want `dX` and `X` has shape [5 x 10], then the only way of achieving this is with `dS.dot(W.T)`, as shown above.

++Note: above tips only work on Loss Functions (scalar output)

In [6]:
import numpy as np

# forward pass
X = np.random.randn(5, 10)  # (N, D)
W = np.random.randn(10, 3)  # (D, C)
S = X.dot(W)                # (N, C)

# now suppose we had the gradient on S from above in the circuit
# dS: dL/dS, dX: dL/dX, dW: dL/dW
dS = np.random.randn(*S.shape) # same shape as S
dX = dS.dot(W.T) #.T gives the transpose of the matrix
dW = X.T.dot(dS)

## **Numpy**

### Max vs. Maximum

In [14]:
import numpy as np

a = np.array([[1, 2], [5, 6]])
b = np.array([3, 4])

print(np.max(a))
print()
print(np.maximum(a, b))

6

[[3 4]
 [5 6]]


### Mask (Boolean array indexing)

In [2]:
import numpy as np

a = np.array([[1,2], [3, 4], [5, 6]])
bool_ind = a > 2
print(bool_ind)
print()
a[a>2] = 0
print(a)

[[False False]
 [ True  True]
 [ True  True]]

[[1 2]
 [0 0]
 [0 0]]


### Slicing vs. Indexing
To index into a matrix, use `np.arrange()` instead of `:`

In [16]:
import numpy as np

a = np.array([[1,2], [3, 4], [5, 6]])

b = a[:, [1, 0, 1]]
print(b)
print()

c = a[np.arange(3), [1, 0, 1]]
print(c)
print()

d = a[[0, 1, 2], [1, 0, 1]]
print(d)

[[2 1 2]
 [4 3 4]
 [6 5 6]]

[2 3 6]

[2 3 6]


### Indexing

In [8]:
import numpy as np

a = np.array([[1, 2], [3, 4]])
b = np.array([[0], [1], [0]])

print(a[b])

[[[1 2]]

 [[3 4]]

 [[1 2]]]


### Broadcast

In [9]:
import numpy as np

x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)

x = x.reshape((2, 1, 3, 4, 4))

print((x * w).shape)

(2, 3, 3, 4, 4)


### Modification to reshaped matrix
It works when the matrix is not a slice of another matrix

In [80]:
import numpy as np

b = np.ones((3, 2, 4, 4))

# c is a slice of b
c = b[:, :, 0:2, 0:2]
print(type(c))
c.reshape((6, 4))[np.arange(6), np.array([0, 2, 1, 0, 2, 1])] += 1
print(c)
print()

d = np.ones((3, 2, 2, 2))
print(type(d))
d.reshape((6, 4))[np.arange(6), np.array([0, 2, 1, 0, 2, 1])] += 1
print(d)

<class 'numpy.ndarray'>
[[[[1. 1.]
   [1. 1.]]

  [[1. 1.]
   [1. 1.]]]


 [[[1. 1.]
   [1. 1.]]

  [[1. 1.]
   [1. 1.]]]


 [[[1. 1.]
   [1. 1.]]

  [[1. 1.]
   [1. 1.]]]]

<class 'numpy.ndarray'>
[[[[2. 1.]
   [1. 1.]]

  [[1. 1.]
   [2. 1.]]]


 [[[1. 2.]
   [1. 1.]]

  [[2. 1.]
   [1. 1.]]]


 [[[1. 1.]
   [2. 1.]]

  [[1. 2.]
   [1. 1.]]]]


## PyTorch

### `view()`, `reshape()`, `permute()`, `transpose()`, and `contiguous()`

- There are a few operations on Tensors in PyTorch that do not change the contents of a tensor, but change the way the data is organized. These operations include:
    
    `narrow()`, `view()`, `expand()` and `transpose()`

In [12]:
import torch
a = torch.rand(1, 2, 3, 4)

b = a.permute(0, 2, 1, 3).view(1, 3, 8)                 # underlying data is still in shape (1, 2, 3, 4)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [13]:
b = a.permute(0, 2, 1, 3).reshape(1, 3, 8)              # use reshape instead
c = a.permute(0, 2, 1, 3).contiguous().view(1, 3, 8)    # or make a contiguous copy before view

c = a.transpose(1, 2).contiguous().view(1, 3, 8)        # transpose is like permute but can only permute 2 dimensions

In [1]:
import torch

print(torch.arange(0, 10, 2))

tensor([0, 2, 4, 6, 8])
